# Classification: Cats VS Dogs

We downloaded the dataset from kaggle. The training archive contains 25,000 images of dogs and cats. 

# Using own data with included Datasets

In [1]:
import os
import mxnet as mx
import matplotlib.pyplot as plt
from mxnet import gluon
from mxnet import nd, autograd

In [2]:
def aug_transform(data, label):
    data = data.astype('float32')/255 # normalize the data
    size= 224
    aug = mx.image.ForceResizeAug((size,size)) # resize the image to 224x224
    data = aug(data)
    data=data.transpose((2, 0, 1))
    return data, label

In [3]:
# x = nd.random_normal(shape=(10,224,224,3))
# y = aug_transform(x,x)

In [4]:
data_folder = "data"
dataset_name = "CatsVsDogs"

training_path = os.path.join(data_folder, dataset_name)

train_dataset = mx.gluon.data.vision.datasets.ImageFolderDataset(training_path,transform=aug_transform)

In [5]:
sample_idx = 17990
sample = train_dataset[sample_idx]
data = sample[0]
label = sample[1]

#plt.imshow(data.asnumpy(), cmap='gray')
print("Data type: {}".format(data.shape))
print("Label: {}".format(label))
print("Label description: {}".format(train_dataset.synsets[label]))
assert label == 1

Data type: (3, 224, 224)
Label: 1
Label description: dog


# Batch size and DataLoader for train_data

In [15]:
batch_size = 128
train_data = gluon.data.DataLoader(train_dataset, batch_size, shuffle=True)

# The AlexNet architecture

In [16]:
alex_net = gluon.nn.Sequential()
with alex_net.name_scope():
    #  First convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=96, kernel_size=11, strides=(4,4), activation='relu'))
    alex_net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    #  Second convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=192, kernel_size=5, activation='relu'))
    alex_net.add(gluon.nn.MaxPool2D(pool_size=3, strides=(2,2)))
    # Third convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=384, kernel_size=3, activation='relu'))
    # Fourth convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=384, kernel_size=3, activation='relu'))
    # Fifth convolutional layer
    alex_net.add(gluon.nn.Conv2D(channels=256, kernel_size=3, activation='relu'))
    alex_net.add(gluon.nn.MaxPool2D(pool_size=3, strides=2))
    # Flatten and apply fullly connected layers
    alex_net.add(gluon.nn.Flatten())
    alex_net.add(gluon.nn.Dense(4096, activation="relu"))
    alex_net.add(gluon.nn.Dense(4096, activation="relu"))
    alex_net.add(gluon.nn.Dense(10))

# Initialize parameters

In [17]:
ctx = mx.gpu()
alex_net.collect_params().initialize(mx.init.Xavier(magnitude=2.24), ctx=ctx)

# Optimizer 

In [18]:
trainer = gluon.Trainer(alex_net.collect_params(), 'sgd', {'learning_rate': .001})

# Softmax cross-entropy loss

In [19]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()

# Evaluation Loop

In [20]:
def evaluate_accuracy(data_iterator, net):
    acc = mx.metric.Accuracy()
    for d, l in data_iterator:
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        output = net(data)
        predictions = nd.argmax(output, axis=1)
        acc.update(preds=predictions, labels=label)
        return acc.get()[1]

# Training loop

In [21]:
epochs = 1
smoothing_constant = .01
metric = mx.metric.Accuracy()

for e in range(epochs):
    metric.reset()
    for i, (d, l) in enumerate(train_data):
        data = d.as_in_context(ctx)
        label = l.as_in_context(ctx)
        with autograd.record():
            output = alex_net(data)
            loss = softmax_cross_entropy(output, label)
        
        loss.backward()
        trainer.step(data.shape[0])
                
        ##########################
        #  Keep a moving average of the losses
        ##########################
        metric.update([label], [output])
        curr_loss = nd.mean(loss).asscalar()
        moving_loss = (curr_loss if ((i == 0) and (e == 0))
                       else (1 - smoothing_constant) * moving_loss + (smoothing_constant) * curr_loss)

        
        if i % 100 == 0 and i > 0:
                name, acc = metric.get()
                print('[Epoch %d Batch %d] Loss: %s Training: %s=%f'%(e, i, moving_loss, name, acc))

        _, train_accuracy = metric.get()
        print("Epoch %s. Loss: %s, Train_acc %s\n" % (e, moving_loss, train_accuracy))


Epoch 0. Loss: 2.302673, Train_acc 0.0

Epoch 0. Loss: 2.3026653957366943, Train_acc 0.0

Epoch 0. Loss: 2.302648757266998, Train_acc 0.0

Epoch 0. Loss: 2.3026248727483747, Train_acc 0.0

Epoch 0. Loss: 2.3025930612386034, Train_acc 0.0046875

Epoch 0. Loss: 2.302552350581662, Train_acc 0.033854166666666664

Epoch 0. Loss: 2.3025056907919708, Train_acc 0.07924107142857142

Epoch 0. Loss: 2.3024510671192195, Train_acc 0.1279296875

Epoch 0. Loss: 2.3023884447613208, Train_acc 0.1675347222222222

Epoch 0. Loss: 2.302318523593432, Train_acc 0.2015625

Epoch 0. Loss: 2.302241228784133, Train_acc 0.2315340909090909

Epoch 0. Loss: 2.302156464792648, Train_acc 0.2571614583333333

Epoch 0. Loss: 2.302065300516273, Train_acc 0.2782451923076923

Epoch 0. Loss: 2.301966460045443, Train_acc 0.2935267857142857

Epoch 0. Loss: 2.301859898548626, Train_acc 0.3104166666666667

Epoch 0. Loss: 2.3017458720500175, Train_acc 0.326171875

Epoch 0. Loss: 2.301625797496235, Train_acc 0.3382352941176471

Ep

Epoch 0. Loss: 2.2584392155676523, Train_acc 0.4815504807692308

Epoch 0. Loss: 2.2578835509434088, Train_acc 0.4817509541984733

Epoch 0. Loss: 2.2573249314221338, Train_acc 0.4823626893939394

Epoch 0. Loss: 2.2567699359111657, Train_acc 0.48214285714285715

Epoch 0. Loss: 2.256210431475455, Train_acc 0.482334421641791

Epoch 0. Loss: 2.2556461079605667, Train_acc 0.4826388888888889

Epoch 0. Loss: 2.2550832625082498, Train_acc 0.4824793198529412

Epoch 0. Loss: 2.2545155360194893, Train_acc 0.48266423357664234

Epoch 0. Loss: 2.253950444574547, Train_acc 0.48278985507246375

Epoch 0. Loss: 2.253381751018999, Train_acc 0.4829136690647482

Epoch 0. Loss: 2.252819864966329, Train_acc 0.4825892857142857

Epoch 0. Loss: 2.2522468560215607, Train_acc 0.4824911347517731

Epoch 0. Loss: 2.2516728203837086, Train_acc 0.4824493838028169

Epoch 0. Loss: 2.2510984025131235, Train_acc 0.48240821678321677

Epoch 0. Loss: 2.250527015617814, Train_acc 0.48193359375

Epoch 0. Loss: 2.249952196854763